In [1]:
import yggdrasilDecisionForests.yggdrasil_decision_forests as ydf


treeple1 environment

In [2]:
! pip install ydf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 41.5 MB/s eta 0:00:00a 0:00:01


In [3]:
import ydf

In [4]:
! pip install pandas

  Using cached pandas-2.2.3-cp310-cp310-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp310-cp310-macosx_11_0_arm64.whl (11.3 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)


In [5]:
import treeple
from treeple.datasets import make_trunk_classification
import pandas as pd
import time
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [6]:
def prepare_dataframe(X, y):
    df = pd.DataFrame(X)
    df.columns = [str(i) for i in df.columns]  # Convert column names to strings
    df["target"] = y.astype(int)  # Append target column

    return df

def train_ydf(X_train, y_train, X_test, y_test, params_ydf, n_dim=500):
    df_train = prepare_dataframe(X_train, y_train)
    df_test = prepare_dataframe(X_test, y_test)

    learner = ydf.RandomForestLearner(**params_ydf)
    start_time = time.time()
    ydf_model = learner.train(df_train)
    time_ydf = time.time() - start_time
    y_pred = ydf_model.predict(df_test)
    y_pred_class = (y_pred >= 0.5).astype(int) 

    acc_ydf = accuracy_score(y_test, y_pred_class)

    print(f"YDF | n_dim: {n_dim} | n_tree: {params_ydf['num_trees']} | Accuracy: {acc_ydf:.4f} | Train Time: {time_ydf:.4f} sec")
    return acc_ydf, time_ydf, y_pred_class

In [7]:
MAX_DEPTH = 10
N_ESTIMATORS = 300
RANDOM_SEED = 42
N_JOBS=-1
BOOTSTRAP = True
MAX_FEATURE = 100
FEATURE_COMBINATIONS = 1.0

params_ydf = dict()
params_ydf["label"] = "target"
params_ydf["max_depth"] = MAX_DEPTH
params_ydf["num_trees"] = N_ESTIMATORS
params_ydf["random_seed"] = RANDOM_SEED
params_ydf["honest"] = False
params_ydf["bootstrap_size_ratio"] = 1.0
params_ydf["bootstrap_training_dataset"] = BOOTSTRAP
params_ydf["categorical_algorithm"] = "ONE_HOT"
params_ydf["compute_oob_performances"] = False  #oob_score
params_ydf["compute_oob_variable_importances"] = False  ## oob_score
params_ydf["in_split_min_examples_check"] = True
params_ydf["min_examples"] = 1    #min_samples_leaf
params_ydf["keep_non_leaf_label_distribution"] = True   # not sure
params_ydf["max_num_nodes"] = None
params_ydf["growing_strategy"] = "BEST_FIRST_GLOBAL"
# params_ydf["num_candidate_attributes"] = -1  #0 equivalent to sqrt max_features in treeple
params_ydf["num_candidate_attributes_ratio"] = 1.0
params_ydf["sorting_strategy"] = "IN_NODE" ###
# sparse oblique params
params_ydf["split_axis"] = "SPARSE_OBLIQUE"
params_ydf["sparse_oblique_weights"] = "CONTINUOUS"
params_ydf["sparse_oblique_projection_density_factor"] = FEATURE_COMBINATIONS
params_ydf["sparse_oblique_normalization"] = "MIN_MAX"
params_ydf["sparse_oblique_max_num_projections"] = MAX_FEATURE # modify this, find in sourse code in treeple
params_ydf["sparse_oblique_num_projections_exponent"] = 1.0   #2.0
params_ydf["sampling_with_replacement"] = False

In [8]:
n_samples=2000
num_dim=100
X, y = make_trunk_classification(n_samples=n_samples, n_dim=num_dim, n_informative=600, seed=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

df_train = prepare_dataframe(X_train, y_train)
df_test = prepare_dataframe(X_test, y_test)

/Users/apple/opt/miniconda3/envs/treeple1/lib/python3.10/site-packages/treeple/datasets/hyppo.py:515: RuntimeWarning: Number of informative dimensions {n_informative} must be less than number of dimensions, 100. Setting n_informative to n_dim.
  warnings.warn(


In [9]:
learner = ydf.RandomForestLearner(**params_ydf)
start_time = time.time()
ydf_model = learner.train(df_train)
time_ydf = time.time() - start_time
pred_ydf = ydf_model.predict(df_test)
pred_ydf = (pred_ydf >= 0.5).astype(int) 

acc_ydf = accuracy_score(y_test, pred_ydf)
acc_ydf

Train model on 1600 examples
Model trained in 0:00:02.805525


0.7325

In [15]:
learner.hyperparameter_templates()

{'better_defaultv1': HyperparameterTemplate(name='better_default', version=1, parameters={'winner_take_all': True}, description='A configuration that is generally better than the default parameters without being more expensive.'),
 'benchmark_rank1v1': HyperparameterTemplate(name='benchmark_rank1', version=1, parameters={'winner_take_all': True, 'categorical_algorithm': 'RANDOM', 'split_axis': 'SPARSE_OBLIQUE', 'sparse_oblique_normalization': 'MIN_MAX', 'sparse_oblique_num_projections_exponent': 1.0}, description='Top ranking hyper-parameters on our benchmark slightly modified to run in reasonable time.')}

In [14]:
ydf_model.describe()

In [23]:
ydf_model.save("saved_model/")

In [24]:
ydf_model.data_spec()

columns {
  type: CATEGORICAL
  name: "target"
  categorical {
    number_of_unique_values: 3
    items {
      key: "<OOD>"
      value {
        index: 0
        count: 0
      }
    }
    items {
      key: "1"
      value {
        index: 2
        count: 800
      }
    }
    items {
      key: "0"
      value {
        index: 1
        count: 800
      }
    }
  }
  count_nas: 0
  dtype: DTYPE_INT64
}
columns {
  type: NUMERICAL
  name: "0"
  numerical {
    mean: 0.46212531956545716
    min_value: -3.50901365
    max_value: 3.9430089
    standard_deviation: 1.117228040824666
  }
  count_nas: 0
  dtype: DTYPE_FLOAT64
}
columns {
  type: NUMERICAL
  name: "1"
  numerical {
    mean: 0.33069338914356194
    min_value: -3.55543637
    max_value: 4.29119062
    standard_deviation: 1.0646178961906465
  }
  count_nas: 0
  dtype: DTYPE_FLOAT64
}
columns {
  type: NUMERICAL
  name: "2"
  numerical {
    mean: 0.295455525021207
    min_value: -3.80400229
    max_value: 3.92988133
    stan

In [26]:
ydf_model.get_tree(1)

Tree(root=NonLeaf(value=ProbabilityValue(num_examples=1600.0, probability=[0.5, 0.5]), condition=NumericalSparseObliqueCondition(missing=False, score=0.0572185292840004, attributes=[1, 10, 43], weights=[0.10236883163452148, 0.07610069215297699, -0.059739671647548676], threshold=0.05464193597435951), pos_child=NonLeaf(value=ProbabilityValue(num_examples=808.0, probability=[0.3341584158415842, 0.6658415841584159]), condition=NumericalSparseObliqueCondition(missing=False, score=0.040012843906879425, attributes=[3], weights=[1.0], threshold=0.20827430486679077), pos_child=NonLeaf(value=ProbabilityValue(num_examples=456.0, probability=[0.21710526315789475, 0.7828947368421053]), condition=NumericalSparseObliqueCondition(missing=False, score=0.036476194858551025, attributes=[1], weights=[1.0], threshold=0.6870007514953613), pos_child=NonLeaf(value=ProbabilityValue(num_examples=326.0, probability=[0.1441717791411043, 0.8558282208588958]), condition=NumericalSparseObliqueCondition(missing=False

In [27]:
ydf_model.get_all_trees()

[Tree(root=NonLeaf(value=ProbabilityValue(num_examples=1600.0, probability=[0.5, 0.5]), condition=NumericalSparseObliqueCondition(missing=False, score=0.08932534605264664, attributes=[1], weights=[1.0], threshold=0.49017471075057983), pos_child=NonLeaf(value=ProbabilityValue(num_examples=799.0, probability=[0.29161451814768463, 0.7083854818523154]), condition=NumericalSparseObliqueCondition(missing=False, score=0.048620592802762985, attributes=[3], weights=[1.0], threshold=0.021114662289619446), pos_child=NonLeaf(value=ProbabilityValue(num_examples=514.0, probability=[0.18482490272373542, 0.8151750972762646]), condition=NumericalSparseObliqueCondition(missing=False, score=0.022567879408597946, attributes=[8, 80], weights=[0.07926032692193985, -0.014232546091079712], threshold=0.11129222810268402), pos_child=Leaf(value=ProbabilityValue(num_examples=82.0, probability=[0.024390243902439025, 0.975609756097561])), neg_child=NonLeaf(value=ProbabilityValue(num_examples=432.0, probability=[0.2

In [28]:
! pip install protobuf

In [29]:
from yggdrasil_decision_forests.model import decision_tree_pb2
#from ydf.model import decision_tree_pb2
from google.protobuf import text_format

# Load the model
with open("saved_model/model.pb", "rb") as f:
    model_proto = decision_tree_pb2.DecisionTreeModel()
    model_proto.ParseFromString(f.read())

ImportError: cannot import name 'decision_tree_pb2' from 'ydf.model' (/Users/apple/opt/miniconda3/envs/treeple1/lib/python3.10/site-packages/ydf/model/__init__.py)

In [30]:
from yggdrasil_decision_forests.model.decision_tree import decision_tree_pb2

def load_model(path):
    model = decision_tree_pb2.DecisionTreeModel()
    with open(path, "rb") as f:
        model.ParseFromString(f.read())
    return model

def extract_projections_from_model(model):
    projections = []
    for tree_idx, tree in enumerate(model.decision_trees):
        print(f"\n🌲 Tree {tree_idx}")
        stack = [tree.root]
        while stack:
            node = stack.pop()
            # Check if this node has an oblique condition
            if node.HasField("condition"):
                condition = node.condition
                if condition.HasField("condition") and condition.condition.HasField("oblique_condition"):
                    oblique = condition.condition.oblique_condition
                    attributes = oblique.attributes
                    weights = oblique.weights
                    threshold = oblique.threshold
                    proj = list(zip(attributes, weights))
                    print(f"  📐 Projection: {proj} >= {threshold}")
                    projections.append((proj, threshold))
            # Push children manually if available
            if hasattr(node, "pos_child"):
                stack.append(node.pos_child)
            if hasattr(node, "neg_child"):
                stack.append(node.neg_child)
    return projections

if __name__ == "__main__":
    model_path = "saved_model/model.pb"  # Update path if needed
    model = load_model(model_path)
    projections = extract_projections_from_model(model)
    print(f"\n✅ Extracted {len(projections)} projection matrices.")


AttributeError: module 'yggdrasil_decision_forests.model.decision_tree.decision_tree_pb2' has no attribute 'DecisionTreeModel'